# Day 5: Binary Boarding

You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

    Start by considering the whole range, rows 0 through 127.
    F means to take the lower half, keeping rows 0 through 63.
    B means to take the upper half, keeping rows 32 through 63.
    F means to take the lower half, keeping rows 32 through 47.
    B means to take the upper half, keeping rows 40 through 47.
    B keeps rows 44 through 47.
    F keeps rows 44 through 45.
    The final F keeps the lower of the two, row 44.

The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

    Start by considering the whole range, columns 0 through 7.
    R means to take the upper half, keeping columns 4 through 7.
    L means to take the lower half, keeping columns 4 through 5.
    The final R keeps the upper of the two, column 5.

So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

    BFFFBBFRRR: row 70, column 7, seat ID 567.
    FFFBBBFRRR: row 14, column 7, seat ID 119.
    BBFFBBFRLL: row 102, column 4, seat ID 820.

## Puzzle 1

As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

In [1]:
# Python imports
from pathlib import Path
from typing import List, Tuple

We write a function to parse a single boarding pass and give the row and seat number as a tuple.

In [2]:
def parse_boarding_pass(code: str, numrows: int=128, numcols: int=8) -> Tuple[int]:
    """Return row and seat number tuple for a boarding pass code
    
    :param code:  boarding pass code
    :param numrows:  number of rows on vessel
    :param numrows:  number of seats in each row
    
    This is essentially a pair of binary search, excluding half of our list at
    a time until only a single value is left. It operates first for rows, and
    then for seats (columns).
    """
    rows = list(range(numrows))
    seats = list(range(numcols))
    
    for char in code:
        if char == "F":
            rows = rows[:(len(rows)//2)]
        elif char == "B":
            rows = rows[(len(rows)//2):]
        elif char == "L":
            seats = seats[:(len(seats)//2)]
        elif char == "R":
            seats = seats[(len(seats)//2):]
    
    return (rows[0], seats[0])

The next function calculates the seat ID as defined in the introduction text.

In [3]:
def calc_seat_id(code: str, numrows: int=128, numcols: int=8) -> int:
    """Return seat ID for a boarding pass code
    
    :param code:  boarding pass code
    :param numrows:  number of rows on vessel
    :param numrows:  number of seats in each row
    """
    row, seat = parse_boarding_pass(code)
    return (row * 8) + seat

And we have a function to load and parse boarding passes from a file

In [4]:
def load_passes(fpath: str) -> List[str]:
    with Path(fpath).open("r") as ifh:
        return [_.strip() for _ in ifh.readlines()]

We test these functions on the example data:

In [5]:
parse_boarding_pass("FBFBBFFRLR")

(44, 5)

In [6]:
passes = load_passes("day05_test.txt")
for bpass in passes:
    print(calc_seat_id(bpass))
    

567
119
820


And then solve the real puzzle.

In [7]:
passes = load_passes("day05_data.txt")
pass_ids = [calc_seat_id(_) for _ in passes]
max(pass_ids)

822

## Puzzle 2

Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

This is a straightforward identification of the first missing number in the list (we don't need to consider the front and back of the plane as those values aren't in the list.

In [8]:
def find_seat(numlist: List[int]) -> List[int]:
    """Return IDs of missing seats in the list
    
    :param numlist:  list of seatIDs
    
    We search from the lowest to the highest value in the list, only.
    """
    numlist = sorted(numlist)
    return [seat for seat in range(numlist[0], numlist[-1]+1) if seat not in numlist]

In [9]:
find_seat(pass_ids)

[705]